## Get Packages

In [2]:
import pandas as pd
import numpy as np
import sys
import itertools
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [28]:
from importlib import reload
import mab
reload(mab)

print(mab.mab_test())

Good - Abraham


## Create Synthetic Segment

In [7]:
row_count = 100000
seg_cols = ['gender', 'age',
            'income', 'buyer', 
            'region', 'area',
            'parent']

segments = ["Male", "Female",
            "Young", "Middle Age", "Older",
            "Low Income", "Medium Income", "High Income",
            "Prior Buyer", "First-Time Buyer",
            "North", "West", "South", "East",
            "Urban", "Suburban",
            "Non-Parent", "Parent"]

combo_weights = mab.create_all_combo_weights()
Segment_df = mab.create_synthetic_sample(row_count=row_count)

combo_weights.groupby('variant_assignment').aggregate({'combos_weights': ['mean', 'count', 'std']})

combos_weights                
                             mean count       std
variant_assignment                               
Control                  0.028601   576  0.024408
Variant A                0.149307   576  0.054214
Variant B                0.147690   576  0.049159
Variant C                0.152026   576  0.051708

## Create Outcome Metric

In [10]:
############
## Step 1 ##
############

## Full random assignment for Variant
Segment_df = mab.add_conversion_rates(df=Segment_df, seg_cols=seg_cols, 
                                      segments=segments,all_combos_weights=combo_weights,
                                      print_diagnostics=True)

converted              
                  mean  count    sum
was_modified                        
False         0.032123   4047    130
True          0.210311  95953  20180

In [11]:
#############
## Step #2 ##
#############

# Take the Target group and build an optimization score to determine how ads should be allocated
Segment_df.loc[Segment_df['variant_assignment'] == 'Variant A', 'Variant_a_performance'] = Segment_df['converted']
Segment_df.loc[Segment_df['variant_assignment'] == 'Variant B', 'Variant_b_performance'] = Segment_df['converted']
Segment_df.loc[Segment_df['variant_assignment'] == 'Variant C', 'Variant_c_performance'] = Segment_df['converted']

## Performance Scores all interactions
perf_scores_all_interactions = Segment_df.groupby(seg_cols).agg({'Variant_a_performance': ['mean'],
                                                                 'Variant_b_performance': ['mean'],
                                                                 'Variant_c_performance': ['mean']}).reset_index().droplevel(1, axis = 1)

In [14]:
###################
## Next Audience ##
###################
Segment_df_step2 = mab.create_synthetic_sample(300000)

Segment_df_step2 = mab.assignment_with_optimization(df = Segment_df_step2, prior_performance_scores=perf_scores_all_interactions, 
                                                    seg_cols=seg_cols,method='max', opt_target_size=0.20, learning_weight=2)

In [16]:
## Create performance score
Segment_df_step2 = mab.add_conversion_rates(df=Segment_df_step2, seg_cols=seg_cols, segments=segments,
                                            all_combos_weights=combo_weights, print_diagnostics=True, 
                                            assign_variant=False)

Segment_df_step2.groupby(['target_control','core_membership'], dropna=False).aggregate({'converted': ['min', 'mean','max', 'count', 'std']})

converted               
                  mean   count    sum
was_modified                         
False         0.032428   15018    487
True          0.226137  284982  64445

converted                                
                                     min      mean max   count       std
target_control core_membership                                          
control        NaN                     0  0.032428   1   15018  0.177139
target_opt     No                      0  0.262521   1   54129  0.440008
               Yes                     0  0.323202   1    5396  0.467743
target_org     NaN                     0  0.215079   1  225457  0.410878

In [30]:
###############################
## Let's turn it into a loop ##
###############################

## Let's do 50 Loops, each time increasing the optimized target sample by 1
overall_performance = []
overall_target_performance = []
org_target_performance = []
opt_target_performance = []

steps = 10
row_count = 500000

for i in range(steps):

    if i == 0:
        
        Segment_df = mab.create_synthetic_sample(row_count=row_count)        
        Segment_df = mab.add_conversion_rates(df=Segment_df, seg_cols=seg_cols, segments=segments, all_combos_weights=combo_weights, print_diagnostics=False)

        ## Store results
        overall_performance.append(Segment_df['converted'].mean())
        org_target_performance.append(Segment_df.loc[Segment_df['variant_assignment'] != 'Control', 'converted'].mean())
        overall_target_performance.append(Segment_df.loc[Segment_df['variant_assignment'] != 'Control', 'converted'].mean())
        opt_target_performance.append(None)

        ### For Next Iteration ###
        
        # Take the Target group and build an optimization score to determine how ads should be allocated
        Segment_df.loc[Segment_df['variant_assignment'] == 'Variant A', 'Variant_a_performance'] = Segment_df['converted']
        Segment_df.loc[Segment_df['variant_assignment'] == 'Variant B', 'Variant_b_performance'] = Segment_df['converted']
        Segment_df.loc[Segment_df['variant_assignment'] == 'Variant C', 'Variant_c_performance'] = Segment_df['converted']
        
        ## Performance Scores all interactions
        perf_scores_all_interactions = Segment_df.groupby(seg_cols).agg({'Variant_a_performance': ['mean'],
                                                                         'Variant_b_performance': ['mean'],
                                                                         'Variant_c_performance': ['mean']}).reset_index().droplevel(1, axis = 1)

    if i > 0:
        
        Segment_df_step2 = mab.create_synthetic_sample(row_count=row_count)
        Segment_df_step2 = mab.assignment_with_optimization(df=Segment_df_step2, prior_performance_scores=perf_scores_all_interactions,seg_cols=seg_cols,method='max', opt_target_size=i/100, learning_weight=2)
        Segment_df_step2 = mab.add_conversion_rates(df=Segment_df_step2, seg_cols=seg_cols, segments=segments, all_combos_weights=combo_weights, print_diagnostics=False, assign_variant=False)
        Segment_df_step2 = Segment_df_step2.reset_index(drop=True)
        
        ## Store Results
        overall_performance.append(Segment_df_step2['converted'].mean())
        overall_target_performance.append(Segment_df_step2.loc[Segment_df_step2['target_control'] != 'control', 'converted'].mean())
        org_target_performance.append(Segment_df_step2.loc[Segment_df_step2['target_control'] == 'target_org', 'converted'].mean())
        opt_target_performance.append(Segment_df_step2.loc[Segment_df_step2['target_control'] == 'target_opt', 'converted'].mean())

        ## For Next Iteration ##
        
        # Take the Target group and build an optimization score to determine how ads should be allocated
        Segment_df_step2.loc[Segment_df_step2['variant_assignment'] == 'Variant A', 'Variant_a_performance'] = Segment_df_step2['converted']
        Segment_df_step2.loc[Segment_df_step2['variant_assignment'] == 'Variant B', 'Variant_b_performance'] = Segment_df_step2['converted']
        Segment_df_step2.loc[Segment_df_step2['variant_assignment'] == 'Variant C', 'Variant_c_performance'] = Segment_df_step2['converted']
        
        ## Performance Scores all interactions
        perf_scores_all_interactions = Segment_df_step2[Segment_df_step2['target_control'] != 'control'].groupby(seg_cols).agg({'Variant_a_performance': ['mean'],
                                                                                                                                'Variant_b_performance': ['mean'],
                                                                                                                                'Variant_c_performance': ['mean']}).reset_index().droplevel(1, axis = 1)

    i += 1 
    mab.progress_bar(i, steps, 40)




Pandas Apply:   0%|          | 0/500000 [00:00<?, ?it/s]

/var/folders/gs/5h077mjs31d2125xyvkp2bkh0000gn/T/ipykernel_7932/781033801.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Segment_df.loc[Segment_df['variant_assignment'] == 'Variant A', 'Variant_a_performance'] = Segment_df['converted']
/var/folders/gs/5h077mjs31d2125xyvkp2bkh0000gn/T/ipykernel_7932/781033801.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Segment_df.loc[Segment_df['variant_assignment'] == 'Variant B', 'Variant_b_performance'] = Segment_df['converted']
/var/folders/gs/5h077mjs31d2

Pandas Apply:   0%|          | 0/500000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/500000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/500000 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
final_results = pd.DataFrame({'Overall Performance':overall_performance,
                             'Performance on All Target':overall_target_performance,
                             'Performance on Organic Target':org_target_performance,
                             'Performance on Optimized Target':opt_target_performance,})

final_results.to_csv('Final Results.csv')

In [22]:
### Create Table of Assignments
seg_data = Segment_df_step2[Segment_df_step2['target_control'] != 'target_org']
name = 'Optimized'

org_table = mab.get_variant_assignment_counts(df = Segment_df_step2[Segment_df_step2['target_control'] == 'target_opt'], table_name='Optimized', seg_cols=seg_cols)
opt_table = mab.get_variant_assignment_counts(df = Segment_df_step2[Segment_df_step2['target_control'] == 'target_org'], table_name='Organic', seg_cols=seg_cols)

pd.concat([org_table, opt_table], axis =1 )

Optimized                       Organic                    
                   Variant A Variant B Variant C Variant A Variant B Variant C
gender Female       0.316233  0.339595  0.344171  0.330199  0.333613  0.336188
       Male         0.322111  0.297739  0.380151  0.333034  0.333931  0.333034
age    Middle Age   0.357717  0.297970  0.344313  0.330313  0.332438  0.337249
       Older        0.301730  0.289619  0.408651  0.332012  0.334173  0.333815
       Young        0.299886  0.372482  0.327632  0.332455  0.334665  0.332880
income High         0.387877  0.297828  0.314296  0.332072  0.333804  0.334124
       Low          0.222308  0.321456  0.456235  0.329018  0.335008  0.335975
       Medium       0.339021  0.339392  0.321588  0.333641  0.332536  0.333823
buyer  First-Time   0.285134  0.314532  0.400335  0.330547  0.334165  0.335288
       Prior Buyer  0.355117  0.323961  0.320922  0.332683  0.333370  0.333947
region East         0.369931  0.340257  0.289812  0.332253  0.333479  0.334268
       North        0.320448  0.267040  0.412513  0.331279  0.335335  0.333386
       South        0.376776  0.279765  0.343459  0.328781  0.335599  0.335621
       West         0.213029  0.385640  0.401331  0.334100  0.330738  0.335162
area   Suburban     0.299587  0.333333  0.367079  0.332084  0.334908  0.333007
       Urban        0.339148  0.304510  0.356342  0.331116  0.332607  0.336278
parent Non-Parent   0.339148  0.304510  0.356342  0.331116  0.332607  0.336278
       Parent       0.299587  0.333333  0.367079  0.332084  0.334908  0.333007

In [24]:
mab.get_variant_assignment_counts(df = Segment_df_step2[Segment_df_step2['target_control'] != 'target_org'], table_name='Optimized', seg_cols=seg_cols)

Optimized                    
                   Variant A Variant B Variant C
gender Female       0.316233  0.339595  0.344171
       Male         0.322111  0.297739  0.380151
age    Middle Age   0.357717  0.297970  0.344313
       Older        0.301730  0.289619  0.408651
       Young        0.299886  0.372482  0.327632
income High         0.387877  0.297828  0.314296
       Low          0.222308  0.321456  0.456235
       Medium       0.339021  0.339392  0.321588
buyer  First-Time   0.285134  0.314532  0.400335
       Prior Buyer  0.355117  0.323961  0.320922
region East         0.369931  0.340257  0.289812
       North        0.320448  0.267040  0.412513
       South        0.376776  0.279765  0.343459
       West         0.213029  0.385640  0.401331
area   Suburban     0.299587  0.333333  0.367079
       Urban        0.339148  0.304510  0.356342
parent Non-Parent   0.339148  0.304510  0.356342
       Parent       0.299587  0.333333  0.367079

In [177]:
opt_table.columns[0]

('Organic', 'Variant A')

In [70]:
seg_counts.iloc[:,[1,2,3]].sum(axis=1)


0    3958
1    4089
dtype: int64

In [98]:
seg_counts

,gender,Variant_a_performance,Variant_b_performance,Variant_c_performance
0,Female,0.359272,0.345124,0.295604
1,Male,0.247493,0.339447,0.413059


In [183]:
for i in range(10):
    print("On iteration " + str(i+1) + " out of " + str(10))

On iteration 1 out of 10
On iteration 2 out of 10
On iteration 3 out of 10
On iteration 4 out of 10
On iteration 5 out of 10
On iteration 6 out of 10
On iteration 7 out of 10
On iteration 8 out of 10
On iteration 9 out of 10
On iteration 10 out of 10


In [205]:
org_table.style.format('{:,.0%}').background_gradient(cmap='Blues').set_properties(**{'text-align': 'center'}).set_table_styles([
   {'selector': 'th',
    'props': [
        ('text-align', 'center')
    ]
    }]
)